https://stepik.org/lesson/1576193/step/6

In [ ]:
import torch
import torch.nn as nn

# здесь объявляйте классы BottleneckBlock1 и BottleneckBlock2

class BottleneckBlock1(nn.Module):
    def __init__(self):
        super().__init__()
        # Вход (batch_size, 64, H, W)
        self.bnb_1 = nn.Sequential(
            nn.Conv2d(64, 64, (1, 1), stride=1, padding=0, bias=False),     # (batch_size, 64, H, W)
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, (3, 3), stride=1, padding=1, bias=False),     # (batch_size, 64, H, W)
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64 * 4, (1, 1), stride=1, padding=0, bias=False), # (batch_size, 64 * 4, H, W)
            nn.BatchNorm2d(64 * 4)
        )

        self.bnb_2 = nn.Sequential(
            nn.Conv2d(64, 64 * 4, (1, 1), stride=1, padding=0, bias=False), # (batch_size, 64 * 4, H, W)
            nn.BatchNorm2d(64 * 4)
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.bnb_1(x) + self.bnb_2(x))                     # (batch_size, 64 * 4, H, W)

class BottleneckBlock2(nn.Module):
    def __init__(self):
        super().__init__()
        # Вход (batch_size, 64, H, W)
        self.bnb = nn.Sequential(
            nn.Conv2d(256, 64, (1, 1), stride=1, padding=0, bias=False),    # (batch_size, 64, H, W)
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, (3, 3), stride=1, padding=1, bias=False),     # (batch_size, 64, H, W)
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 256, (1, 1), stride=1, padding=0, bias=False),    # (batch_size, 64 * 4, H, W)
            nn.BatchNorm2d(256)
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.bnb(x) + x)                                   # (batch_size, 64 * 4, H, W)

batch_size = 8
x = torch.rand(batch_size, 3, 32, 32) # тензор x в программе не менять

model = nn.Sequential(
            nn.Conv2d(3, 64, (7, 7), stride=2, padding=3, bias=False), # (batch_size, 64, 16, 16)
            nn.BatchNorm2d(64),
            nn.MaxPool2d((3, 3), stride=2, padding=1),                 # (batch_size, 64, 8, 8)
            BottleneckBlock1(),                                        # (batch_size, 256, 8, 8)
            BottleneckBlock2(),                                        # (batch_size, 256, 8, 8)
            nn.AdaptiveAvgPool2d(1),                                   # (batch_size, 256, 1, 1)
            nn.Flatten(),                                              # (batch_size, 256)
            nn.Linear(256, 10, bias=True)                              # (batch_size, 10)
)

model.eval()
predict = model(x)

# predict.shape

torch.Size([8, 10])